In [3]:
import pandas as pd

#read json into a dataframe
df_idf=pd.read_json("/content/stackoverflow-data-idf.json",lines=True,encoding='utf-8')

# print schema
print("Schema:\n\n",df_idf.dtypes)
print("Number of questions,columns=",df_idf.shape)

Schema:

 id                            int64
title                        object
body                         object
answer_count                  int64
comment_count                 int64
creation_date                object
last_activity_date           object
last_editor_display_name     object
owner_display_name           object
owner_user_id               float64
post_type_id                  int64
score                         int64
tags                         object
view_count                    int64
accepted_answer_id          float64
favorite_count              float64
last_edit_date               object
last_editor_user_id         float64
community_owned_date         object
dtype: object
Number of questions,columns= (20000, 19)


In [4]:
import re
def pre_process(text):

    # lowercase
    text=text.lower()

    #remove tags
    text=re.sub("<!--?.*?-->","",text)

    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)

    return text

df_idf['text'] = df_idf['title'] + df_idf['body']
df_idf['text'] = df_idf['text'].apply(lambda x:pre_process(x))

#show the second 'text' just for fun
df_idf['text'][2]

'gradle command line p i m trying to run a shell script with gradle i currently have something like this p pre code def test project tasks create test exec commandline bash c bash c my file dir script sh code pre p the problem is that i cannot run this script because i have spaces in my dir name i have tried everything e g p pre code commandline bash c bash c my file dir script sh tokenize commandline bash c bash c my file dir script sh commandline bash c new stringbuilder append bash append c my file dir script sh commandline bash c bash c my file dir script sh file dir file c my file dir script sh commandline bash c bash dir getabsolutepath code pre p im using windows bit and if i use a path without spaces the script runs perfectly therefore the only issue as i can see is how gradle handles spaces p '

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

def get_stop_words(stop_file_path):
    """load stop words """

    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))

#load a set of stop words
stopwords=get_stop_words("stopwords.txt")

#get the text column
docs=df_idf['text'].tolist()

#create a vocabulary of words,
#ignore words that appear in 85% of documents,
#eliminate stop words
cv=CountVectorizer(max_df=0.85,stop_words=stopwords,max_features=10000)
word_count_vector=cv.fit_transform(docs)

list(cv.vocabulary_.keys())[:10]

['serializing',
 'private',
 'struct',
 'public',
 'class',
 'contains',
 'properties',
 'string',
 'serialize',
 'attempt']

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [22]:
# read test docs into a dataframe and concatenate title and body
df_test=pd.read_json("stackoverflow-test.json",lines=True)
print("Number of questions,columns=",df_test.shape)
df_test['text'] = df_test['title'] + df_test['body']
df_test['text'] =df_test['text'].apply(lambda x:pre_process(x))

# get test docs into a list
docs_test=df_test['text'].tolist()

Number of questions,columns= (500, 18)


In [20]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""

    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    # word index and corresponding tf-idf score
    for idx, score in sorted_items:

        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]

    return results

In [21]:
# you only needs to do this once, this is a mapping of index to
feature_names=cv.get_feature_names_out()

# get the document that we want to extract keywords from
doc=docs_test[0]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)

# now print the results
print("\n=====Doc=====")
print(doc)
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Doc=====
integrate war plugin for m eclipse into eclipse project p i set up a small web project with jsf and maven now i want to deploy on a tomcat server is there a possibility to automate that like a button in eclipse that automatically deploys the project to tomcat p p i read about a the a href http maven apache org plugins maven war plugin rel nofollow noreferrer maven war plugin a but i couldn t find a tutorial how to integrate that into my process eclipse m eclipse p p can you link me to help or try to explain it thanks p 

===Keywords===
eclipse 0.49
maven 0.451
war 0.393
plugin 0.265
integrate 0.233
tomcat 0.223
project 0.197
automate 0.13
jsf 0.125
possibility 0.121


In [23]:
# get the document that we want to extract keywords from
doc=docs_test[1]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)

# now print the results
print("\n=====Doc=====")
print(doc)
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Doc=====
phantomjs node page evaulate seems to hang p i have an implementation of waitfor with phantomjs node and it seems that the code sitepage evaluate code has a big lag compared to when it should evaluate true you ll see below that i m logging out the content value and the content logs with what should evaluate as true but this doesn t seem to occur for a good seconds or so after the fact p p any idea what would cause this delay or if there s a better way to evaluate p pre code let promise require bluebird let phantom require phantom let sitepage let phinstance phantom create then instance gt phinstance instance return instance createpage then page gt sitepage page return page open https thepiratebay org search game then status gt return waituntil function this returns the correct content after a short period while the evaluate ends up taking maybe s longer after this content should evaluate true sitepage property content then content gt console log content return sitepage e

In [24]:
# get the document that we want to extract keywords from
doc=docs_test[2]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)

# now print the results
print("\n=====Doc=====")
print(doc)
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Doc=====
dynamic operations can only be performed in homogenous appdomain p i m working with an api that requires p pre code lt trust level full legacycasmodel true gt code pre p to be set in my web config file in order to work this works without a problem in vs but when i use vs i get an error stating that dynamic operations can only be performed in homogeneous appdomain the project is targeting net framework and is an asp net mvc project p p i tried changing legacycasmodel to false but then i can t access the object i need not sure how to resolve this issue how can i use legacycasmodel true in vs with dynamic expressions p 

===Keywords===
appdomain 0.41
dynamic 0.384
performed 0.332
operations 0.297
targeting 0.199
trust 0.182
net 0.182
project 0.179
stating 0.178
expressions 0.167


1.   How many samples are in the training set?<br>
There are 20,000 samples in the traning set.
2.   How many samples are in the test set?<br>
There are 500 samples in the test set.
3.   What is this data set?<br>
The data set is about StackOverflow posts, with 19 specific fields such as answer count, body, comment count, title, creation date, tags, score, viewc count, etc. We were interested in the title and body.
4.   Use the code from the tutorial to obtain these answers:  What are the top 10 keywords and their TF-IDF scores from the test set sample 1 (the document about eclipse project)?  How about sample 2 (the one about phantomjs)?  How about sample 3 (the one about dynamic operations)?<br>
     1.  eclipse (0.49), maven (0.451), war (0.393), plugin (0.265), integrate (0.233), tomcat (0.223), project (0.197), automate (0.13), jsf (0.125), possibility (0.121)
     2. evaluate (0.474), content (0.403), console (0.281), log (0.265), function (0.215), promise (0.2), return (0.195), wait (0.169), let (0.163), resolve (0.156)
     3. appdomain (0.41), dynamic (0.384), performed (0.332), operations (0.297), targeting (0.199), trust (0.182), net (0.182), project (0.179), stating (0.178), expressions (0.167)